# TAREFA 1

In [12]:
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup
import requests
import re
import time

def get_pagina(url):
    try:
        response = urlopen(url)
        bs = BeautifulSoup(response, 'html.parser')
    except HTTPError as e:
        print("Houve um erro na obtenção da página! ", e)
        return None
    except URLError as e:
        print("Ocorreu um erro no servidor!", e)
        return None
    else:
        return bs

def salva_arquivo(url):
    pagina = url
    response = requests.get(pagina)
    soup = BeautifulSoup(response.content)
    titulo = soup.select(".mw-page-title-main")
    try:
        nome_arquivo = titulo[0].text
    except IndexError as e:
        print(f'error no index: {e}')
        return
    with open(f"links/{nome_arquivo}.html", "w", encoding="utf-8") as f:
        conteudo = response.content.decode("utf-8")
        f.write(conteudo)

def distribui_links(url):
    bs = get_pagina(url)
    links = bs.find_all('a', href=True)
    salva_arquivo(url)
    visited_links.append(url)
    for link in links:
        if(len(non_visited_links) == 5000):
            print('chegou em 5000')
            break
        #time.sleep(1)
        if re.match(r"^\/wiki\/[^:]+$", link['href']): #verbetes começam com /wiki/ e nao tem dois pontos (:)
            #print(link['href'])
            if link['href'] not in visited_links:
                non_visited_links.append(f'https://pt.wikipedia.org{link['href']}')
                #print('pagina adicionada!')
            else:
                print('################# PAGINA NAO ADICIONADA ##############')

In [13]:
# INICIO DA POPULAÇÃO DAS LISTAS A PARTIR DA HOME DA WIKIPEDIA
visited_links = []
non_visited_links = []
URL_INICIAL = 'https://pt.wikipedia.org'
bs = get_pagina(URL_INICIAL)
links = bs.find_all('a', href=True)
for link in links:
    if re.match(r"^\/wiki\/[^:]+$", link['href']): #verbetes começam com /wiki/ e nao tem dois pontos (:)
        if link['href'] not in visited_links:
            non_visited_links.append(f'https://pt.wikipedia.org{link['href']}')
        else:
            print('################# PAGINA NAO ADICIONADA ##############')
print(len(visited_links))
print(len(non_visited_links))

0
120


In [ ]:
for link in non_visited_links:
    print(f'links salvos = {len(visited_links)}')
    print(f'non visited_links tamanho = {len(non_visited_links)}')
    if link not in visited_links:
        distribui_links(link)
        

links salvos = 0
non visited_links tamanho = 120
links salvos = 1
non visited_links tamanho = 1258
links salvos = 2
non visited_links tamanho = 1448
links salvos = 3
non visited_links tamanho = 1611
links salvos = 4
non visited_links tamanho = 2691
error no index: list index out of range
links salvos = 5
non visited_links tamanho = 2807
links salvos = 5
non visited_links tamanho = 2807
links salvos = 6
non visited_links tamanho = 2870
links salvos = 7
non visited_links tamanho = 2904
links salvos = 8
non visited_links tamanho = 3015
links salvos = 9
non visited_links tamanho = 3300
links salvos = 10
non visited_links tamanho = 3400
links salvos = 11
non visited_links tamanho = 4010
links salvos = 12
non visited_links tamanho = 4928
chegou em 5000
links salvos = 13
non visited_links tamanho = 5000
chegou em 5000
links salvos = 14
non visited_links tamanho = 5000
chegou em 5000
links salvos = 15
non visited_links tamanho = 5000
error no index: list index out of range
chegou em 5000
links

# TAREFA 2

In [10]:
import json
def create_json(url):
    bs = get_pagina(url)
    conteudo_principal = bs.find("div", class_="mw-parser-output")
    tabela = conteudo_principal.find("table", class_="infobox infobox_v2") if conteudo_principal else None
    if(tabela):
        print('lendo', url)
        trs = tabela.find_all("tr")
        titulo = trs[0].text.strip()
        titulo = re.sub(r'[^\w\s-]', '', titulo)  # remove caracteres especiais
        titulo = re.sub(r'\s+', '_', titulo)      # substitui espaços por underline
        print(titulo)
        dicionario = {}
        for tr in trs:
            tds = tr.find_all('td')
            if(len(tds) == 2):
                div = tds[1].find("div")
                if(div):
                    ul = div.find("ul")
                    if(ul):
                        #print(f'TEM LISTAAA!!!! -----> {tds[0].text.strip()}')
                        lis = ul.find_all('li')
                        data_lista = []
                        try: #caso haja paragrafo E TAMBEM lista para a mesma chave
                            data_lista.append(tds[1].span.text.strip())
                        except:
                            continue
                        for li in lis:
                            data_lista.append(li.text.strip())
                            #print(f'colocando {li.text.strip()} na lista {tds[0].text.strip()}')
                        dicionario[tds[0].text.strip()] = data_lista
                else:
                    dicionario[tds[0].text.strip()] = tds[1].text.strip()
        with open(f"jsons/{titulo}.json", 'w') as f:
            json.dump(dicionario, f, indent=2, ensure_ascii=False)

In [11]:
for link in visited_links:
    create_json(link)

lendo https://pt.wikipedia.org/wiki/Escritor
Escritor
lendo https://pt.wikipedia.org/wiki/Maya_Angelou

lendo https://pt.wikipedia.org/wiki/M%C3%BAsica
Música
lendo https://pt.wikipedia.org/wiki/Impeachment_de_Yoon_Suk-yeol
Processo_de_impeachment_de_Yoon_Suk-yeol
lendo https://pt.wikipedia.org/wiki/Corte_Constitucional_da_Coreia
Corte_Constitucional_da_Coreia_대한민국_헌법재판소
lendo https://pt.wikipedia.org/wiki/Yoon_Suk-yeol
Yoon_Suk-yeol윤석열
lendo https://pt.wikipedia.org/wiki/Donald_Trump
Donald_Trump
lendo https://pt.wikipedia.org/wiki/For%C3%A7as_Armadas_do_Sud%C3%A3o
Forças_Armadas_do_Sudão
